In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf

In [3]:
import csv

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Common imports
import numpy as np
import os

import tensorflow as tf


# to make this notebook's output stable across runs
# def reset_graph(seed=42):
#     tf.reset_default_graph()
#     tf.set_random_seed(seed)
#     np.random.seed(seed)

import time
start = time.time()

print ('PRIPREMA PODATAKA')
vege_csv = csv.reader(open('vegetacija.csv', newline=''), delimiter=' ', quotechar='|')

vege_array = []


for row in vege_csv:
	red_array = []
	items = row[0].split(';')
	for i in items:
		red_array.append(i)

	vege_array.append(red_array)


klase = []
values = []

vege_array_fix = vege_array[:-4]

print ('length of the fixed array is ', len (vege_array_fix))

for r in vege_array_fix:
    value_row = []
    i = 0
    while i < len(r):
        if i == 0:
            if r[i] == 'X':
                klase.append(0)
            if r[i] == 'V':
                klase.append(1)
            if r[i] == 'N':
                klase.append(2)
        else:
            value_row.append(int(r[i]))
        i = i + 1
    values.append(value_row)

print ('test : ', vege_array[10000])

PRIPREMA PODATAKA
length of the fixed array is  199400
test :  ['V', '339', '201', '238', '278', '186', '298', '298', '253']


In [4]:
print (type(vege_array[9000]))

<class 'list'>


In [5]:
print (klase[7777])

1


In [6]:
print (values[7777])

[358, 220, 262, 305, 213, 302, 289, 245]


In [7]:

print ('KREIRANJE GRAFA')
reset_graph()

train_x, test_x, train_y, test_y = train_test_split(values, klase, test_size=0.20)


# n_inputs = 8 
# n_hidden1 = 20
# n_hidden2 = 10
# n_hidden3 = 5
# n_outputs = 3

KREIRANJE GRAFA


In [8]:

# he_init = tf.variance_scaling_initializer()
# hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
#                           kernel_initializer=he_init, name="hidden1")

In [9]:
print ('dataset size is ', len(vege_array_fix))

print ('x size is ', len(train_x))
print ('y size is ', len(train_y))

print ('x test size is ', len(test_x))
print ('y test size is ', len(test_y))

dataset size is  199400
x size is  159520
y size is  159520
x test size is  39880
y test size is  39880


In [10]:
train_x[3]

[303, 165, 184, 197, 124, 257, 235, 203]

In [11]:
train_x = np.asarray(train_x)
train_x = train_x.astype(np.float32).reshape(-1, 8)/1000
test_x = np.asarray(test_x)
test_x = test_x.astype(np.float32).reshape(-1, 8)/1000

train_y = np.asarray(train_y)
test_y = np.asarray(test_y)
train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)

X_valid, X_train =  train_x[:15952], train_x[15952:]
y_valid, y_train =  train_y[:15952], train_y[15952:]


In [12]:
filter1 = [2,2,3,3,3,3,2,2]
filter2 = [5,5,3,3,3,3,5,5]
filter3 = [7,7,7,7,2,2,2,2]
filter4 = [3,3,3,3,1,1,1,1]


#dio za conv1d

batch_size = 1280       # Batch size
seq_len = 64          # Number of steps
learning_rate = 0.001
epochs =100

n_classes = 3
n_channels = 8
#

tf.reset_default_graph()

# X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
# y = tf.placeholder(tf.int32, shape=(None), name="y")

# conv1 = tf.layers.conv1d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
#                          strides=conv1_stride, padding=conv1_pad,
#                          activation=tf.nn.relu, name="conv1")
# conv2 = tf.layers.conv1d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
#                          strides=conv2_stride, padding=conv2_pad,
#                          activation=tf.nn.relu, name="conv2")

# with tf.name_scope("inputs_for_cnn"):
inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
labels_ = tf.placeholder(tf.float32, [None, 1], name = 'labels')
keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

In [13]:
#Build Convolutional Layers

with tf.name_scope("cnn"):
    # (batch, 64, 8) --> (batch, 32, 16)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=16, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 32, 16) --> (batch, 16, 16)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=16, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 16, 16) --> (batch, 8, 32)
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=32, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    # (batch, 8, 32) --> (batch, 4, 32)
    conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=32, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')


In [14]:
#Build the inception layer

with tf.name_scope('inception'):
    # convolution: input to output of inception (size=1)
    # (batch, 4, 32) --> (batch, 4, 32)
    conv1_11 = tf.layers.conv1d(inputs=max_pool_4, filters=32, kernel_size=1, strides=1,
                               padding='same', activation = tf.nn.relu)
    # convolution: input to middle layer of inception (size=1)
    # (batch, 4, 32) --> (batch, 4, 16)
    conv1_21 = tf.layers.conv1d(inputs=max_pool_4, filters=16, kernel_size=1, strides=1,
                               padding='same', activation = tf.nn.relu)
    
    # convolution: input to middle layer of inception (size=1)
    # (batch, 4, 32) --> (batch, 4, 16)
    conv1_31 = tf.layers.conv1d(inputs=max_pool_4, filters=16, kernel_size=1, strides=1,
                               padding='same', activation = tf.nn.relu)
    
    # average pool: input to middle layer of inception
    # (batch, 4, 32) --> (batch, 4, 32)
    avg_pool_41 = tf.layers.average_pooling1d(inputs=max_pool_4, pool_size=2, strides=1, padding='same')
    
    ## Middle layer of inception
    
    # convolution: middle to out layer of inception (size=2)
    # (batch, 4, 16) --> (batch, 4, 32)
    conv2_22 = tf.layers.conv1d(inputs=conv1_21, filters=32, kernel_size=2, strides=1,
                               padding='same', activation=tf.nn.relu)
    
    # convolution: middle to out layer of inception (size=4)
    # (batch, 4, 16) --> (batch, 4, 32)
    conv4_32 = tf.layers.conv1d(inputs=conv1_31, filters=32, kernel_size=4, strides=1,
                               padding='same', activation=tf.nn.relu)
    
    # convolution: middle to out layer of inception (size=1)
    # (batch, 4, 32) --> (batch, 4, 32)
    conv1_42 = tf.layers.conv1d(inputs=avg_pool_41, filters=32, kernel_size=1, strides=1,
                               padding='same', activation=tf.nn.relu)
    
    ## Out layer: Concatenate filters
    # (batch, 4, 2*32)
    inception_out = tf.concat([conv1_11, conv2_22, conv4_32, conv1_42], axis=2)

In [15]:
#Now, flatten and pass to the classifier

with tf.name_scope('flatten_and_pass'):
    # Flatten and add dropout
    flat = tf.reshape(inception_out, (-1, 4*64))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
    # Predictions
    logits = tf.layers.dense(flat, n_classes)
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [16]:
print (len(X_train))

X_train = X_train[:143360]
y_train = y_train[:143360]
y_train
len(y_train)

143568


143360

In [17]:
def get_batches(X, y, batch_size):
	""" Return a generator for batches """
	n_batches = len(X) // batch_size
	X, y = X[:n_batches*batch_size], y[:n_batches*batch_size]

	# Loop over batches and yield
	for b in range(0, len(X), batch_size):
		yield X[b:b+batch_size], y[b:b+batch_size]

In [18]:
# #train the network

# validation_acc = []
# validation_loss = []

# train_acc = []
# train_loss = []

# with tf.name_scope('saver'):
#     saver = tf.train.Saver()

# with tf.Session() as sess:
#     #slijedecu liniju mozda promijeniti
#     sess.run(tf.global_variables_initializer())
#     iteration = 1
   
#     # Loop over epochs
#     for e in range(epochs):
        
#         # Loop over batches
#         for x,y in get_batches(X_train, y_train, batch_size):
            
#             # Feed dictionary
#             feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            
#             # Loss
#             loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
#             train_acc.append(acc)
#             train_loss.append(loss)
            
#             # Print at each 5 iters
#             if (iteration % 5 == 0):
#                 print("Epoch: {}/{}".format(e, epochs),
#                       "Iteration: {:d}".format(iteration),
#                       "Train loss: {:6f}".format(loss),
#                       "Train acc: {:.6f}".format(acc))
            
#             # Compute validation loss at every 10 iterations
#             if (iteration%10 == 0):                
#                 val_acc_ = []
#                 val_loss_ = []
                
#                 for x_v, y_v in get_batches(X_valid, y_valid, batch_size):
#                     # Feed
#                     feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  
                    
#                     # Loss
#                     loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
#                     val_acc_.append(acc_v)
#                     val_loss_.append(loss_v)
                
#                 # Print info
#                 print("Epoch: {}/{}".format(e, epochs),
#                       "Iteration: {:d}".format(iteration),
#                       "Validation loss: {:6f}".format(np.mean(val_loss_)),
#                       "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
#                 # Store
#                 validation_acc.append(np.mean(val_acc_))
#                 validation_loss.append(np.mean(val_loss_))
            
#             # Iterate 
#             iteration += 1
    
#     saver.save(sess,"checkpoints-cnn/vege.ckpt")


In [19]:
X_train_r = X_train.reshape(int(len(X_train)/64), 64, 8)
y_train_r = y_train.reshape(int(len(y_train)), 1)

In [20]:
#train the network

validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with tf.name_scope('saver'):
    saver = tf.train.Saver()

with tf.Session() as sess:
    #slijedecu liniju mozda promijeniti
    sess.run(tf.global_variables_initializer())
    iteration = 1
   
    # Loop over epochs
    for e in range(epochs):

        # Feed dictionary
        feed = {inputs_ : X_train_r, labels_ : y_train_r, keep_prob_ : 0.5, learning_rate_ : learning_rate}

        # Loss
        loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
        train_acc.append(acc)
        train_loss.append(loss)

        # Print at each 5 iters
        if (iteration % 5 == 0):
            print("Epoch: {}/{}".format(e, epochs),
                  "Iteration: {:d}".format(iteration),
                  "Train loss: {:6f}".format(loss),
                  "Train acc: {:.6f}".format(acc))

        # Compute validation loss at every 10 iterations
        if (iteration%10 == 0):                
            val_acc_ = []
            val_loss_ = []

            for x_v, y_v in get_batches(X_valid, y_valid, batch_size):
                # Feed
                feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  

                # Loss
                loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                val_acc_.append(acc_v)
                val_loss_.append(loss_v)

            # Print info
            print("Epoch: {}/{}".format(e, epochs),
                  "Iteration: {:d}".format(iteration),
                  "Validation loss: {:6f}".format(np.mean(val_loss_)),
                  "Validation acc: {:.6f}".format(np.mean(val_acc_)))

            # Store
            validation_acc.append(np.mean(val_acc_))
            validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints-cnn/vege.ckpt")


InvalidArgumentError: logits and labels must be broadcastable: logits_size=[4480,3] labels_size=[143360,1]
	 [[node flatten_and_pass/softmax_cross_entropy_with_logits_sg (defined at <ipython-input-15-7cc5f41bdeeb>:12)  = SoftmaxCrossEntropyWithLogits[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](flatten_and_pass/dense/BiasAdd, flatten_and_pass/softmax_cross_entropy_with_logits_sg/Reshape_1)]]
	 [[{{node flatten_and_pass/accuracy/_13}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_986_flatten_and_pass/accuracy", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'flatten_and_pass/softmax_cross_entropy_with_logits_sg', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, 